In [23]:
import itertools
import collections

class NumberOfWells:
    allBases = {"A", "T", "C", "G"}
    basesMatch = {"A": "T", "C": "G", "G": "C", "T": "A"}
    letterToInt = {"A": "1", "C": "2", "G": "3", "T": "4"}
#lengthOfSequence - 4 = "ATCG"
#differing Length - if true, will include all sequences >1 and <4, i.e. all sequences of 2,3 as well
#minMatch - how many bases have to be pairing in order for it to be 
#consecutive, whether or not they have to be considered consecutive    
    def __init__(self, lengthOfSequences = 4, differingLength = False, minMatch = 4, consecutive = True):
        assert(minMatch <= lengthOfSequences)
        assert(minMatch > 1)
        self.lengthOfSequences = lengthOfSequences
        self.differingLength = differingLength
        self.minMatch = minMatch
        self.consecutive = consecutive
        self.allPossibleSequences = []
        self.sumIfAllMatching = ((10**(self.lengthOfSequences) - 1) * (5))//9
        if self.differingLength:
            for x in range(2, self.lengthOfSequences + 1):
                self.allPossibleSequences+=list(itertools.product("1234", repeat = x))
        else:
            self.allPossibleSequences = list(itertools.product("1234", repeat = self.lengthOfSequences))
        print(len(self.allPossibleSequences))
        self.allPossibleSequences = list(map(lambda x: int("".join(x)), self.allPossibleSequences))
        self.findAllStrandsMatch()
        
    #will convert 3' to 5' to 5' to 3' and vice versa   
    def reverseStrand(self, strand):
        reverse = 0
        while(strand > 0):
            remainder = strand %10
            reverse = (reverse *10) + remainder
            strand = strand//10
        return reverse
    
    #Takes 3' to 5' strand and returns 3' to 5' strand
    def findComplement(self, strand1):
        return self.reverseStrand(self.sumIfAllMatching - strand1)
    
    #Takes 3' to 5' strand and 3' to 5' strand 
    def ifTwoStrandsMatch(self, strand1, strand2):
        maxStagger = self.lengthOfSequences - self.minMatch + 1 #Used to help with the offset
        sequenceToSearchFor = "5"*self.minMatch
        for i in range(maxStagger):
            original1 = self.reverseStrand(strand1) + int(strand2)*(10**i) #5 to 3 and 3 to 5
            primer1 = self.reverseStrand(self.findComplement(strand1)) + int(strand2)*(10**i) #5 to 3 and 3 to 5
            original2 = int(strand1)*(10**i) + self.reverseStrand(strand2)  #3 to 5 and 5 to 3
            primer2 = self.reverseStrand(self.findComplement(strand2)) + int(strand1)*(10**i) #5 to 3 and 3 to 5
            allTestStrands = [str(original1), str(primer1), str(original2), str(primer2)]
            if not self.consecutive:
                allTestStrands = list(map(lambda x: "".join(sorted(x)), allTestStrands))
            if any(list(map(lambda x: sequenceToSearchFor in x, allTestStrands))):
                return True
        return False
    
    def findAllStrandsMatch(self):
        self.allMatchingStrands = collections.defaultdict(set)
        for index1 in range(0, (len(self.allPossibleSequences)//2)):
            x = self.allPossibleSequences[index1]
            for index2 in range(index1, (len(self.allPossibleSequences)//2)):
                y = self.allPossibleSequences[index2]
                if self.ifTwoStrandsMatch(x,y):
                    self.allMatchingStrands[x].add(y)
                    self.allMatchingStrands[x].add(self.findComplement(y))
                    self.allMatchingStrands[y].add(x)
                    self.allMatchingStrands[y].add(self.findComplement(x))
                    self.allMatchingStrands[self.sumIfAllMatching - x].add(y)
                    self.allMatchingStrands[self.sumIfAllMatching - x].add(self.findComplement(y))
                    self.allMatchingStrands[self.sumIfAllMatching - y].add(x)
                    self.allMatchingStrands[self.sumIfAllMatching - y].add(self.findComplement(x))
                    
    def findAllDisjointIndependentSets(self):
        allSets = []
        allMatchingStrands = self.allMatchingStrands.copy()
        while len(allMatchingStrands):
            tempSet = set()
            allMatching = set()
            for strand in allMatchingStrands:
                if strand not in allMatching:
                    tempSet.add(strand)
                    allMatching.update(allMatchingStrands[strand])
            for strand in tempSet:
                allMatchingStrands.pop(strand)
            allSets.append(tempSet)
        self.allSets = allSets

In [24]:
testcase = NumberOfWells(lengthOfSequences = 5,minMatch = 4, consecutive = False)
testcase.findAllDisjointIndependentSets()
print(len(testcase.allSets))

1024
19


In [25]:
print(testcase.allSets)

[{31232, 34311, 34312, 31241, 34313, 31242, 34321, 34322, 34323, 12311, 34331, 34332, 34333, 33311, 33312, 34341, 34342, 33321, 33322, 33331, 33332, 32311, 32312, 33341, 33342, 32321, 32322, 32331, 32332, 31311, 44111, 44112, 31312, 32341, 32342, 44121, 44122, 31321, 31322, 44131, 44132, 44133, 44134, 43111, 43112, 31331, 31332, 34411, 34412, 44141, 44142, 44143, 34413, 43121, 43122, 31341, 31342, 34421, 34422, 34423, 43131, 43132, 42111, 34432, 42112, 34431, 33411, 33412, 43141, 43142, 34441, 34442, 42121, 42122, 33421, 33422, 42131, 42132, 41111, 41112, 33431, 33432, 32411, 32412, 42141, 42142, 41121, 41122, 33441, 33442, 32421, 32422, 41131, 41132, 32431, 32432, 31411, 44211, 41141, 44214, 44213, 44212, 41142, 31412, 32441, 32442, 44221, 44222, 44223, 44224, 31421, 31422, 44231, 44232, 44233, 44234, 43211, 43212, 31431, 31432, 44241, 44242, 44243, 31441, 43221, 43222, 31442, 43231, 43232, 42211, 42212, 43241, 43242, 42221, 42222, 42231, 42232, 41211, 41212, 42241, 42242, 41221, 4122